In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd '/content/drive/MyDrive/Colab_Notebooks/MIDI_transformer'

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/MIDI_transformer


In [2]:
!pip install --upgrade keras-nlp
!pip install --upgrade pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Suc

In [3]:
import keras
from midi_utils.preprocessing import Tokenizer
from midi_utils.train import generate_examples

#from importlib import reload
#reload(midi_utils)

from midi_utils.train import generate_examples

In [5]:
tokenizer = Tokenizer()
tokenizer_file = 'training_data/english_pieces/6_bars_aug/tokenizer.json'
tokenizer.load(tokenizer_file)

In [6]:
model = keras.saving.load_model('models/model_v9/model.keras')

In [8]:
examples = generate_examples('examples', model, tokenizer, 350)

sampled token: EVEN
sampled token: NOTE=57
sampled token: DELTA=4.0
sampled token: NOTE=61
sampled token: DELTA=4.0
sampled token: NOTE=45
sampled token: DELTA=3.0
sampled token: NOTE=64
sampled token: DELTA=2.0
sampled token: STEP
sampled token: DELTA=2.0
sampled token: NOTE=65
sampled token: DELTA=2.0
sampled token: STEP
sampled token: DELTA=1.0
sampled token: NOTE=47
sampled token: DELTA=1.0
sampled token: STEP
sampled token: DELTA=1.0
sampled token: MEASURE
sampled token: EVEN
sampled token: NOTE=49
sampled token: DELTA=2.0
sampled token: NOTE=57
sampled token: DELTA=2.0
sampled token: NOTE=64
sampled token: DELTA=2.0
sampled token: STEP
sampled token: DELTA=1.0
sampled token: NOTE=69
sampled token: DELTA=1.0
sampled token: STEP
sampled token: DELTA=1.0
sampled token: NOTE=50
sampled token: DELTA=2.0
sampled token: NOTE=62
sampled token: DELTA=2.0
sampled token: NOTE=71
sampled token: DELTA=2.0
sampled token: STEP
sampled token: DELTA=2.0
sampled token: MEASURE
sampled token: EVEN


In [ ]:
from midi_utils.encoding_decoding import decode_tokens_to_midi
from midi_utils.midi_generation import generate_midi

def find_faulty_measure(events):

    current_time = 0
    expected_time = 0
    measure_index = 0

    for index, event in enumerate(events):
        if events[index-1] == 'STEP':
            _, delta = event.split('=')
            delta = float(delta)
            current_time += delta

        if event == 'EVEN':
            if current_time - expected_time != 0:
                return measure_index
            measure_index = index - 1
            expected_time = 4
            current_time = 0

        if event == 'TRIPLE':
            if current_time - expected_time != 0:
                if current_time - expected_time != 0:
                    return measure_index
            measure_index = index - 1
            expected_time = 3
            current_time = 0

    if current_time - expected_time != 0:
        return measure_index
    else:
        return index + 1


def find_second_measure(events):
    measure_counter = 0
    for index, event in enumerate(events):
        if event == 'MEASURE':
            measure_counter +=1
        if measure_counter == 2:
            return index
    raise ValueError('Input sequence must have at least two measures.')


def create_new_input(events):

    second_measure_index = find_second_measure(events)
    print(second_measure_index)
    faulty_measure_index = find_faulty_measure(events)
    print(faulty_measure_index)

    first_measure = events[:second_measure_index]
    new_input = events[second_measure_index:faulty_measure_index]

    return first_measure, new_input


def generate_measures(model, tokenizer, seed, num_gens, max_len, proba, tempo=100, save_midi=False, output_path=None):

    generated_events = generate_midi(
        model=model,
        tokenizer=tokenizer,
        seed=seed,
        max_len=max_len,
        p=proba,
        tempo=100,
        save_midi=False
    )[0]

    piece = []
    for i in range(num_gens):

        first_measure, new_input = create_new_input(generated_events)
        piece.append(first_measure)

        generated_events = generate_midi(
            model=model,
            tokenizer=tokenizer,
            seed=new_input,
            max_len=max_len,
            p=proba,
            tempo=100,
            save_midi=False
        )[0]

    piece.append(generated_events)
    final_piece = [event for part in piece for event in part]

    midi_object = decode_tokens_to_midi(final_piece, tempo=tempo)

    if save_midi:
        midi_object.write(output_path)

    return final_piece, midi_object

In [ ]:
seed = ['MEASURE', 'EVEN', 'NOTE=43', 'DELTA=2.0', 'NOTE=58', 'DELTA=1.5', 'NOTE=62', 'DELTA=1.5', 'NOTE=67', 'DELTA=1.5', 'STEP', 'DELTA=1.0', 'NOTE=55', 'DELTA=0.5',
        'STEP', 'DELTA=0.5', 'NOTE=57', 'DELTA=0.5', 'NOTE=65', 'DELTA=0.5', 'STEP', 'DELTA=0.25', 'NOTE=63', 'DELTA=0.25', 'STEP', 'DELTA=0.25', 'NOTE=43', 'DELTA=2.0',
        'NOTE=58', 'DELTA=0.5', 'NOTE=62', 'DELTA=0.5', 'STEP', 'DELTA=0.5', 'NOTE=60', 'DELTA=0.5', 'STEP', 'DELTA=0.5', 'NOTE=62', 'DELTA=1.0', 'NOTE=70', 'DELTA=1.0',
        'STEP', 'DELTA=1.0', 'MEASURE', 'EVEN', 'NOTE=51', 'DELTA=2.0', 'NOTE=55', 'DELTA=2.0', 'NOTE=63', 'DELTA=1.0', 'STEP', 'DELTA=0.5', 'NOTE=72', 'DELTA=0.25',
        'STEP', 'DELTA=0.25', 'NOTE=70', 'DELTA=0.25', 'STEP', 'DELTA=0.25', 'NOTE=69', 'DELTA=0.5', 'STEP', 'DELTA=0.5', 'NOTE=67', 'DELTA=0.5']

In [ ]:
for prob in [0.7, 0.75, 0.8]:

    for i in range(2):

        output_path = f'examples/output_piece_prob_{prob}_{i+1}.mid'

        generate_measures(
            model=model,
            tokenizer=tokenizer,
            seed=seed,
            num_gens=10,
            max_len=350,
            proba=prob,
            tempo=120,
            save_midi=True,
            output_path=output_path
        )

sampled token: STEP
sampled token: DELTA=0.5
sampled token: NOTE=50
sampled token: DELTA=2.0
sampled token: NOTE=57
sampled token: DELTA=2.0
sampled token: NOTE=66
sampled token: DELTA=0.75
sampled token: STEP
sampled token: DELTA=0.5
sampled token: NOTE=62
sampled token: DELTA=0.25
sampled token: STEP
sampled token: DELTA=0.25
sampled token: NOTE=64
sampled token: DELTA=0.25
sampled token: STEP
sampled token: DELTA=0.25
sampled token: NOTE=66
sampled token: DELTA=0.25
sampled token: STEP
sampled token: DELTA=0.25
sampled token: NOTE=67
sampled token: DELTA=0.25
sampled token: STEP
sampled token: DELTA=0.25
sampled token: NOTE=69
sampled token: DELTA=0.25
sampled token: STEP
sampled token: DELTA=0.25
sampled token: NOTE=70
sampled token: DELTA=0.25
sampled token: STEP
sampled token: DELTA=0.25
sampled token: MEASURE
sampled token: EVEN
sampled token: NOTE=38
sampled token: DELTA=2.0
sampled token: NOTE=62
sampled token: DELTA=1.0
sampled token: NOTE=69
sampled token: DELTA=0.5
sampled 

In [ ]:
output_directory = 'examples'
generate_examples(output_directory, model, tokenizer, max_len=350)
